# Train a better model

In [ ]:
from theano.sandbox import cuda

In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using TensorFlow backend.


NameError: name 'reload' is not defined

In [ ]:
path = 'data/dogscat/'
model_path = path _ 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size = 64

## Are we underfitting?

If we use all the data to train the model, you will find that the val_accuracy >= train_accuracy. That leads to 2 obvious questions:

1. Why/How is this possible?
2. Is this desirable?

**Answer for 1)**

Because of _dropout_. Dropouts refer to a layer that randomlyl deletes (i.e. sets to zero) each activation in the previous layer with probability _p_.
> p is generally set to 0.5

This only happens during training, not when calculating the accuracy on the validation set, which is why the validation set can show higher accuracy than the training set. 

The purpose of dropout is to avoid overfitting. By deleting parts of the neural netowrk at random during training, it ensures that no one part of the netowrk can overfit to one part of the training set. 
> The create of dropout was one of the key developments in deep learning

It has allowed us to create rihc models w/o overfitting. However, it can also result in underfitting if overused, and this is sth we should be careful of with our model.

**Answer for 2)**

This is probably not desirable. It is likely that we can get better validation set results with less dropout. 
> Validation accuracy is higher thatn training accuracy? ==> A strong sign of underfitting!!

So let's remove dropout entirely, and see waht happens


### Removing dropout

I will start with the fine-tuned cats vs dogs model (with dropout). Then fine-tune all the dense layers, without dropout.

Action Plan:
* Re-create and load our modified VGG model with binary dependent (dogs vs cats)
* Split the model between the convoluntional layers and the dense layers
* Pre-calculate the output of the conv layers, so that we don't have to redundently calculate them on every epoch
* Create a new model with just the dense layers, and dropout p set to 0
* Train this new model using the output of the cov layers as training data

In [ ]:
# Step 1
model = vgg_ft(2)
model.load_weights(model_path+'finetune3.h5')

We're going to be training a number of iterations without dropout, so it would be best for us to pre-calculate the input to the fully connected layers - i.e. the Flatten() layer. We'll start by finding this layer in our model, and creating a new model that contains just the layers up to and including this layer:

> Dense layers are also known as fully connected layers

In [ ]:
layers = model.layers
last_conv_idx = [index for index, layer in enumerate(layers)
                if type(layer) is Convolution2D][-]
last_conv_idx

In [ ]:
layers[last_conv_idx]

In [ ]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
fc_layers = layers[last_conv_idx+1:]